In [ ]:
# Data

# Loan table
loan_id,
loan_amount,
loan_term,
applicant_id,
loan_status

# Applicant_Detail
applicant_id
no_of_dependents,
education_id,
self_employed_id,
income_annum,
cibil_score,
residential_assets_value,
commercial_assets_value,
luxury_assets_value,
bank_asset_value,

# Education
education_id,
education

# Employment
self_employed_id,
self_employed

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
base_folder = "/content/drive/MyDrive/MS/Python Project"
%cd "{base_folder}"

/content/drive/MyDrive/MS/Python Project


In [19]:
from pathlib import Path
import os
import sqlite3
import pandas as pd
import tarfile
import urllib.request

def load_housing_data():
  return pd.read_csv(Path("data/loan_approval_dataset.csv"), skipinitialspace=True)


def build_3nf_sqlite(db_path="data/housing.db"):
  print("=== BUILDING 3NF SQLITE DATA MODEL ===")

  loan = load_housing_data()
  print(f"Loaded {len(loan)} rows.")
  print(loan.columns)

  loan['applicant_id'] = loan.index + 1

  education_dim = (
        loan[["education"]]
        .drop_duplicates()
        .reset_index(drop=True)
  )

  education_dim["education_id"] = education_dim.index + 1

  print("\n[STEP 4] Merging education_id into main DataFrame…")
  loan = loan.merge(education_dim, on="education", how="left")

  employment_dim = (
        loan[["self_employed"]]
        .drop_duplicates()
        .reset_index(drop=True)
  )

  employment_dim["self_employed_id"] = employment_dim.index + 1

  print("\n[STEP 4] Merging self_employed_id into main DataFrame…")
  loan = loan.merge(employment_dim, on="self_employed", how="left")

  print("\n[STEP 5] Creating 3NF DataFrames (ocean, block, stats)…")
  df_education = education_dim.rename(columns={"education": "name"})[
        ["education_id", "name"]
  ]

  df_employment = employment_dim.rename(columns={"self_employed": "flag"})[
        ["self_employed_id", "flag"]
  ]

  df_loan = loan[
        ["loan_id", "loan_amount", "loan_term", "applicant_id", "loan_status"]
    ]
  df_applicant = loan[
        [
            "applicant_id",
            "no_of_dependents",
            "education_id",
            "self_employed_id",
            "income_annum",
            "cibil_score",
            "residential_assets_value",
            "commercial_assets_value",
            "luxury_assets_value",
            "bank_asset_value"
        ]
    ]
  print("3NF DataFrames created.")

  # print(df_education.head())
  # print(df_employment.head())
  # print(df_loan.head())
  # print(df_applicant.head())

  if os.path.exists(db_path):
        print("Existing DB found. Removing…")
        os.remove(db_path)

  conn = sqlite3.connect(db_path)
  cur = conn.cursor()

  print("Running SQL schema creation script…")
  cur.executescript(
        """
        DROP TABLE IF EXISTS loan;
        DROP TABLE IF EXISTS applicant;
        DROP TABLE IF EXISTS education;
        DROP TABLE IF EXISTS employment;

        CREATE TABLE education (
            education_id  INTEGER PRIMARY KEY,
            name                TEXT NOT NULL UNIQUE
        );

        CREATE TABLE employment (
            self_employed_id  INTEGER PRIMARY KEY,
            flag                TEXT NOT NULL UNIQUE
        );

        CREATE TABLE loan (
            loan_id           INTEGER PRIMARY KEY,
            loan_amount          REAL NOT NULL,
            loan_term           REAL NOT NULL,
            applicant_id        INTEGER NOT NULL,
            loan_status         TEXT NOT NULL,
            FOREIGN KEY (applicant_id)
                REFERENCES applicant(applicant_id)
        );

        CREATE TABLE applicant (
            applicant_id            INTEGER PRIMARY KEY,
            no_of_dependents  REAL NOT NULL,
            education_id         INTEGER NOT NULL,
            self_employed_id      INTEGER NOT NULL,
            income_annum          REAL NOT NULL,
            cibil_score          REAL NOT NULL,
            residential_assets_value   REAL NOT NULL,
            commercial_assets_value   REAL NOT NULL,
            luxury_assets_value   REAL NOT NULL,
            bank_asset_value   REAL NOT NULL
        );
        """
    )
  print("Tables created.")

  print("\n[STEP 7] Inserting data into SQLite database…")

  print("Inserting education table…")
  cur.executemany(
        "INSERT INTO education (education_id, name) VALUES (?, ?)",
        list(df_education.itertuples(index=False, name=None)),
    )

  print("Inserting employment table…")
  cur.executemany(
        "INSERT INTO employment (self_employed_id, flag) VALUES (?, ?)",
        list(df_employment.itertuples(index=False, name=None)),
    )

  print("Inserting loan table…")
  cur.executemany(
        """
        INSERT INTO loan (loan_id, loan_amount, loan_term, applicant_id,loan_status)
        VALUES (?, ?, ?, ?, ?)
        """,
        list(df_loan.itertuples(index=False, name=None)),
    )

  print("Inserting applicant…")
  cur.executemany(
        """
        INSERT INTO applicant (
            applicant_id,
            no_of_dependents,
            education_id,
            self_employed_id,
            income_annum,
            cibil_score,
            residential_assets_value,
            commercial_assets_value,
            luxury_assets_value,
            bank_asset_value
        )
        VALUES (?, ?, ?, ?, ?, ?, ?, ?,?,?)
        """,
        list(df_applicant.itertuples(index=False, name=None)),
    )

  conn.commit()
  conn.close()

  print("\n=== DONE! SQLite DB created at:", db_path, "===\n")



build_3nf_sqlite("data/housing.db")




=== BUILDING 3NF SQLITE DATA MODEL ===
Loaded 4269 rows.
Index(['loan_id', 'no_of_dependents', 'education', 'self_employed',
       'income_annum', 'loan_amount', 'loan_term', 'cibil_score',
       'residential_assets_value', 'commercial_assets_value',
       'luxury_assets_value', 'bank_asset_value', 'loan_status'],
      dtype='object')

[STEP 4] Merging education_id into main DataFrame…

[STEP 4] Merging self_employed_id into main DataFrame…

[STEP 5] Creating 3NF DataFrames (ocean, block, stats)…
3NF DataFrames created.
Existing DB found. Removing…
Running SQL schema creation script…
Tables created.

[STEP 7] Inserting data into SQLite database…
Inserting education table…
Inserting employment table…
Inserting loan table…
Inserting applicant…

=== DONE! SQLite DB created at: data/housing.db ===



In [22]:
import sqlite3
import pandas as pd
conn = sqlite3.connect(f"{base_folder}/data/housing.db")

loan = pd.read_sql_query(
    """
    SELECT
        c.no_of_dependents,
        d.name as education,
        e.flag as self_employed,
        c.income_annum,
        b.loan_amount,
        b.loan_term,
        c.cibil_score,
        c.residential_assets_value,
        c.commercial_assets_value,
        c.luxury_assets_value,
        c.bank_asset_value,
        b.loan_status

    FROM loan AS b
    LEFT JOIN applicant AS c
        ON b.applicant_id = c.applicant_id
    LEFT JOIN education AS d
        ON c.education_id = d.education_id
    LEFT JOIN employment AS e
        ON c.self_employed_id = e.self_employed_id
    ORDER BY b.applicant_id
    """,
    conn,
)
conn.close()

print(loan.count())

no_of_dependents            4269
education                   4269
self_employed               4269
income_annum                4269
loan_amount                 4269
loan_term                   4269
cibil_score                 4269
residential_assets_value    4269
commercial_assets_value     4269
luxury_assets_value         4269
bank_asset_value            4269
loan_status                 4269
dtype: int64
